# Chatbot v0.1 Según enfermedad o IMC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk #herramientas de lenguaje natural
import numpy as np # Para la estructuras de datos y matrices)
import random
import string #usar y procesar cadenas de texto
#nuevas librerias---
import torch #libreria para el aprendizaje automatico, utiliza los tensorflow, ejecuta el codigo de forma nativa usando la GPU
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#base de datos de dieta
f=open('/content/basededatosNut.json','r',errors = 'ignore')

raw=f.read()
raw=raw.lower()# convertir a minusculas

sent_tokens = nltk.sent_tokenize(raw)# convierte a lista de oraciones 
word_tokens = nltk.word_tokenize(raw)# convierte a una lista de palabras

lemmer = nltk.stem.WordNetLemmatizer()
#WordNet es un diccionario de inglés de orientación semántica incluido en NLTK.


#lematizar palabras
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens] 


remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation) #Puntuacion de palabras 

#print(remove_punct_dict)


def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))



#ENTRADAS Y RESPUESTAS PREDETERMINADAS
GREETING_INPUTS = ("Hola")
GREETING_RESPONSES = [ "Bienvenido", "Hola"]

#funcion para una respuesta predeterminada 
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)
        
#SKLEARN biblioteca para aprendizaje automático
from sklearn.feature_extraction.text import TfidfVectorizer

#Matrices de palabras similares
from sklearn.metrics.pairwise import cosine_similarity


#funcion de procesamiento
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens) #vectoriza la palabras
    vals = cosine_similarity(tfidf[-1], tfidf) #Calcule la similitud de coseno entre muestras en X e Y.
    idx=vals.argsort()[0][-2] #compara las similitudes en mun margen de dos items
    flat = vals.flatten() #Devuelve una copia de la matriz contraída en una dimensión.
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Disculpa, no te comprendo"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

flag=True



print("BotNut: Mi nombre es BotNut. Responderé a tus consultas sobre dietas. Si desea salir, escriba ¡Adiós!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='adios'):
        if(user_response=='gracias' or user_response=='gracias a ti' ):
            flag=False
            print("BotNut: de nada..")
        else:
            if(greeting(user_response)!=None):
                print("BotNut: "+greeting(user_response))
            else:
                print("BotNut: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("BotNut: ¡Adiós! Cuídate..")

BotNut: Mi nombre es BotNut. Responderé a tus consultas sobre dietas. Si desea salir, escriba ¡Adiós!
saludo
BotNut: 

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


{
  "converzacion": [
    {
      "etiqueta": "saludo",
      "patrones": [
        "hola",
        "oye",
        "cómo estás",
        "¿hay alguien ahí?
gracias
BotNut: de nada..


#Chatbot v2


In [ ]:
import nltk #herramientas de lenguaje natural
import numpy as np #Para la estructuras de datos y matrices
import random #herramienta para generar números pseudoaleatorios
import json #herramienta para el intercambio de datos y para analizar los datos

import torch #libreria para el aprendizaje automatico, utiliza los tensorflow, ejecuta el codigo de forma nativa usando la GPU
import torch.nn as nn #Permite construir redes neuronales
from torch.utils.data import Dataset, DataLoader #Permite indicar el conjunto de datos que que se cargará

from nltk_utils import bag_of_words, tokenize, stem
from modelo import NeuralNet #contiene las funciones de torch redes neuronales

nltk.download('punkt') #Este tokenizer se entrenó bien para trabajar con muchos idiomas.

#cargamos nuestra bd
with open('basededatos.json', 'r') as f:
    intenciones = json.load(f)

todas_las_palabras = []
etiquetas = []
xy = []
# recorrer cada oración en nuestros patrones de intenciones
for inten in intenciones['intenciones']:
    etiqueta = inten['etiqueta']
    # agregar a la lista de etiquetas
    etiquetas.append(etiqueta)
    for patron in inten['patrones']:
        # tokenizar cada palabra de la oración
        w = tokenize(patron)
        # agregar a nuestra lista de palabras
        todas_las_palabras.extend(w)
        # agregar al par xy
        xy.append((w, etiqueta))

# ignorar palabra
ignorar_palabras = ['?', '.', '!']
todas_las_palabras = [stem(w) for w in todas_las_palabras if w not in ignorar_palabras]
# eliminar duplicados y ordenar
todas_las_palabras = sorted(set(todas_las_palabras))
etiquetas = sorted(set(etiquetas))

print(len(xy), "patrones")
print(len(etiquetas), "etiquetas:", etiquetas)
print(len(todas_las_palabras), "palabras derivadas unicas:", todas_las_palabras)

# crear datos de entrenamiento
X_cola = []
Y_cola = []
for (patron_de_oracion, etiqueta) in xy:
    # X: bolsa de palabras para cada patron de sentencia
    bolsa = bag_of_words(patron_de_oracion, todas_las_palabras)
    X_cola.append(bolsa)
    # y: PyTorch CrossEntropyLoss solo necesita etiquetas de clase, no one-hot
    ETIQUETA_ = etiquetas.index(etiqueta)
    Y_cola.append(ETIQUETA_)

X_cola = np.array(X_cola)
Y_cola = np.array(Y_cola)

# Hiperparámetros
interaciones = 1000 #NUMERO DE INTERACIONES
batch_size = 8      #tamanio de lote
tasa_de_aprendizaje = 0.001
tamanio_de_entrada = len(X_cola[0])
tamanio_oculto = 8
tamanio_de_salida = len(etiquetas)
print(tamanio_de_entrada, tamanio_de_salida)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_cola)
        self.x_data = X_cola
        self.y_data = Y_cola

    # Admite la indexación de modo que el conjunto de datos [i] se pueda utilizar para obtener la i-ésima muestra
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # podemos llamar a len (conjunto de datos) para devolver el tamaño
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
cargador_de_cola = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

dispositivo = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

modelo = NeuralNet(tamanio_de_entrada, tamanio_oculto, tamanio_de_salida).to(dispositivo)

# Pérdida y optimizacion
criterio = nn.CrossEntropyLoss()
optimizador = torch.optim.Adam(modelo.parameters(), lr=tasa_de_aprendizaje)

# Entrena el modelo
for nivel in range(interaciones):
    for (PALABRAS, ETIQUETAS) in cargador_de_cola:
        PALABRAS = PALABRAS.to(dispositivo)
        ETIQUETAS = ETIQUETAS.to(dtype=torch.long).to(dispositivo)
        
        # Pase adelantado
        SALIDAS = modelo(PALABRAS)
        # si y sería one-hot, debemos aplicar
        # etiquetas = antorcha.max (etiquetas, 1) [1]
        PERDIDAS = criterio(SALIDAS, ETIQUETAS)
        
        # Retroceder y optimizar
        optimizador.zero_grad()
        PERDIDAS.backward()
        optimizador.step()
        
    if (nivel+1) % 100 == 0:
        print (f'interaciones [{nivel+1}/{interaciones}], perdidos: {PERDIDAS.item():.4f}')


print(f'perdida final: {PERDIDAS.item():.4f}')

datos = {
"estado_del_modelo": modelo.state_dict(),
"tamanio_de_entrada": tamanio_de_entrada,
"tamanio_oculto": tamanio_oculto,
"tamanio_de_salida": tamanio_de_salida,
"todas_las_palabras": todas_las_palabras,
"etiquetas": etiquetas
}

FILE = "datos.pth"
torch.save(datos, FILE)

print(f'Entrenamiento completado, archivo guardado en: {FILE}')

In [ ]:
import random
import json

import torch #libreria para el aprendizaje automatico, utiliza los tensorflow, ejecuta el codigo de forma nativa usando la GPU

from modelo import NeuralNet
from nltk_utils import bag_of_words, tokenize

#declaramos el objeto torch
dispositivo = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#cargamos nuestra bd
with open('basededatos.json', 'r') as json_data:
    INTENCIONES = json.load(json_data)

#cargamos losdatos de entrenamiento
FILE = "datos.pth"
datos = torch.load(FILE)

#obtenemos los paraetros
_tamanio_entrada = datos["tamanio_de_entrada"]
_tamanio_oculto = datos["tamanio_oculto"]
_tamanio_salida = datos["tamanio_de_salida"]
_todas_las_palabras = datos['todas_las_palabras']
_etiqueta = datos['etiquetas']
_estado_modelo = datos["estado_del_modelo"]

#crea el modelo de reconocimiento
MODELO = NeuralNet(_tamanio_entrada, _tamanio_oculto, _tamanio_salida).to(dispositivo)
MODELO.load_state_dict(_estado_modelo)
MODELO.eval() #evalua las respuestas


bot_name = "PsicoRobo"
print(f"{bot_name}: Mi nombre es PsicoRobo. Responderé a tus consultas, si desea salir, escriba adios")
while True:
    sentencias = input("Tu: ")
    if sentencias == "adios":
        print(f"{bot_name}:Que tengas un buen día")
        break

    sentencias = tokenize(sentencias) #tokenizamos las palabras
    X = bag_of_words(sentencias, _todas_las_palabras) #coincidencia de palabras con la bd json
    X = X.reshape(1, X.shape[0]) 
    X = torch.from_numpy(X).to(dispositivo) #Objeto de entrenamiento y posibles respuestas

    salida = MODELO(X) 
    _, prediccion = torch.max(salida, dim=1)

    ETIQUETA = _etiqueta[prediccion.item()] #etiquetas y predicciones

    problemas = torch.softmax(salida, dim=1) #entrada de n dimensiones y rescalado de salidas de n dimensiones
    probabilidad = problemas[0][prediccion.item()] #prediccion de la respuesta mas acertada
    if probabilidad.item() > 0.75: #probabilidades mayores a 75%
        for intenc in INTENCIONES['intenciones']: #posibles intenciones
            if ETIQUETA == intenc["etiqueta"]:
                print(f"{bot_name}: {random.choice(intenc['respuestas'])}")
    else:
        print(f"{bot_name}: No entiendo...")